In [ ]:

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets,transforms

import matplotlib.pyplot as plt


## Transform

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.2,0.2,0.2))
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.2,0.2,0.2))
])


## CIFAR10 dataset

In [ ]:
train_data = torchvision.datasets.CIFAR10(root='./data',train=True, download=True, transform = train_transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

test_data = torchvision.datasets.CIFAR10(root='./data',train=False, download=True, transform = test_transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)



## Model

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        self.conv1 = nn.Conv2d(3,32,3)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32,64,3)

        self.fc1 = nn.Linear(64*6*6, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        print("init ",x.shape)
        x = self.pool(torch.relu(self.conv2(x)))
        print("after 1 conv,pooling",x.shape)
        x = self.pool(torch.relu(self.conv2(x)))
        print("after 2 conv,pooling",x.shape)

        x = torch.flatten(x,1)
        x = torch.relu(self.fc1(x))
        print("after fc1", x.shape)
        x = self.fc2(x)
        print("after fc2", x.shape)


## Train/Test

In [ ]:
def trian_eval(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    #training
    num_epochs=5
    for epoch in range(num_epochs):
        for i, data in enumerate(train_loader,0):
            images,labels = data

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            if (epoch+1)%10 ==0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Error:{loss.item()}")

    print('Finish training')
    correct = 0
    total = 0

    with torch.no_grad():
        for (images, labels) in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total+= labels.size(0)
            correct += (predicted==labels).sum().item()

    acc = 100* correct/total
    print(f"Accuarcy: {acc:.2f}")


In [ ]:

print("LeNet-5")
model = LeNet()
trian_eval(model)

#파라미터수
print("LaNet",sum(p.numel() for p in model.parameters()))
